In [1]:
# import all relevant libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import requests
import json
from sec_api import QueryApi

import yfinance as yf

I am using base code from https://gist.github.com/janlukasschroeder/3da274150fd00c1c1776c7e541a9b61b to get me started in scraping financial statements from the SEC. I will be using 10-K forms (annual SEC filing) rather than all quarterly filings (10-Q) to reduce the size of data.

In [9]:
from sec_api import QueryApi

api_key = '88140c3a554da7220ba7453a5e1ea2221c5f1e0e42c3293bbe75857245774e2e'

# get your API key at https://sec-api.io
query_api = QueryApi(api_key=api_key)

# fetch all 10-K filings for Apple from the last 25 years to test this API
query = {
    "query": {
        "query_string": {
            "query": "filedAt:[2011-01-01 TO 2012-01-01] AND (formType:\"10-K\")"
        }
    },
    "from": "0",
    "size": "100",
    "sort": [{ "filedAt": { "order": "desc" } }]
}

query_result_100 = query_api.get_filings(query)

In [10]:
# XBRL-to-JSON converter API endpoint
xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"

In [11]:
query_result

NameError: name 'query_result' is not defined

In [12]:
query_result_100

{'total': {'value': 10000, 'relation': 'gte'},
 'query': {'from': 0, 'size': 100},
 'filings': [{'id': '53be96c6c4517dfc6465bc2f0ab7b2aa',
   'accessionNo': '0001144204-11-072464',
   'cik': '1435530',
   'ticker': '',
   'companyName': 'China Polypeptide Group, Inc.',
   'companyNameLong': 'China Polypeptide Group, Inc. (Filer)',
   'formType': 'NT 10-K',
   'description': 'Form NTN 10K - Notices of Late Filings of Form 10-K or 10-KSB',
   'filedAt': '2011-12-30T21:03:48-05:00',
   'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1435530/000114420411072464/0001144204-11-072464.txt',
   'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/1435530/0001144204-11-072464-index.htm',
   'linkToXbrl': '',
   'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1435530/000114420411072464/v244435_nt10k.htm',
   'entities': [{'companyName': 'China Polypeptide Group, Inc. (Filer)',
     'cik': '0001435530',
     'irsNo': '208731646',
     'stateOfIncorporation': 'DE',
     'fis

In [14]:
sample_company = query_result_100['filings'][0]

In [15]:
acc_no = sample_company['accessionNo']
acc_no

'0001144204-11-072464'

In [16]:
accession_numbers = []

# extract accession numbers of each filing
for filing in query_result_100['filings']:
    accession_numbers.append(filing['accessionNo']);

accession_numbers

['0001144204-11-072464',
 '0001513162-11-000410',
 '0001393905-11-000854',
 '0001471242-11-000511',
 '0001354488-11-005251',
 '0001185185-11-002321',
 '0001013762-11-003420',
 '0001513162-11-000404',
 '0001144204-11-072364',
 '0001144204-11-072359',
 '0001078782-11-003814',
 '0001354488-11-005245',
 '0000930413-11-008132',
 '0001354488-11-005242',
 '0001102624-11-000811',
 '0001004878-11-000435',
 '0001437749-11-009914',
 '0001078782-11-003810',
 '0001437749-11-009912',
 '0001019687-11-003992',
 '0001437749-11-009909',
 '0001513162-11-000401',
 '0001079973-11-001036',
 '0001165527-11-001255',
 '0000072633-11-000030',
 '0001264931-11-000743',
 '0001091818-11-000653',
 '0001078782-11-003808',
 '0001078782-11-003805',
 '0001437749-11-009896',
 '0001213900-11-006882',
 '0001214659-11-004557',
 '0001165527-11-001252',
 '0001038838-11-000399',
 '0001127855-11-000735',
 '0001127855-11-000733',
 '0001144204-11-072245',
 '0001391609-11-000602',
 '0001078782-11-003800',
 '0001140361-11-059222',


In [17]:
import time

# get XBRL-JSON for a given accession number
def get_xbrl_json(accession_no, retry = 0):
    request_url = xbrl_converter_api_endpoint + "?accession-no=" + accession_no + "&token=" + api_key

    # linear backoff in case API fails with "too many requests" error
    try:
      response_tmp = requests.get(request_url)
      xbrl_json = json.loads(response_tmp.text)
    except:
      if retry > 5:
        raise Exception('API error')
      
      # wait 500 milliseconds on error and retry
      time.sleep(0.5) 
      return get_xbrl_json(accession_no, retry + 1)

    return xbrl_json

In [34]:
sample_xbrl = get_xbrl_json(query_result_100['filings'][24]['accessionNo'])

In [35]:
sample_xbrl

{'status': 404, 'message': 'No XBRL data found for this filing.'}

In [36]:
list_xbrl

NameError: name 'list_xbrl' is not defined

In [37]:
list_xbrl = []

for acc_no in accession_numbers:
    list_xbrl.append(get_xbrl_json(acc_no))

In [38]:
cleaned_list_xbrl = []

for xbrl in list_xbrl:
    if 'status' in xbrl.keys():
        pass
    else:
        cleaned_list_xbrl.append(xbrl)

In [39]:
len(cleaned_list_xbrl)

37

In [40]:
len(list_xbrl)

100

In [59]:
cleaned_list_xbrl[19]

{'CoverPage': {'AmendmentFlag': 'false',
  'CurrentFiscalYearEndDate': '--09-30',
  'DocumentFiscalPeriodFocus': 'FY',
  'DocumentFiscalYearFocus': '2011',
  'DocumentPeriodEndDate': '2011-09-30',
  'DocumentType': '10-K',
  'EntityCentralIndexKey': '0000869495',
  'EntityCommonStockSharesOutstanding': {'unitRef': 'shares',
   'decimals': 'INF',
   'period': {'instant': '2011-11-25'},
   'value': '136739971'},
  'EntityCurrentReportingStatus': 'Yes',
  'EntityFilerCategory': 'Smaller Reporting Company',
  'EntityPublicFloat': {'unitRef': 'iso4217_USD',
   'decimals': '0',
   'period': {'instant': '2011-03-31'},
   'value': '9800000'},
  'EntityRegistrantName': 'DEEP WELL OIL & GAS INC',
  'EntityVoluntaryFilers': 'No',
  'EntityWellKnownSeasonedIssuer': 'No',
  'TradingSymbol': 'DWOG'},
 'BalanceSheets': {'AccountsPayableCurrent': [{'unitRef': 'iso4217_USD',
    'decimals': '0',
    'period': {'instant': '2010-09-30'},
    'value': '42147'},
   {'unitRef': 'iso4217_USD',
    'decimals'

In [41]:
#### convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    

        income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 

    income_statement = pd.DataFrame(income_statement_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return income_statement.T 



In [21]:
get_balance_sheet(sample_xbrl)

,2008-09-27,2009-09-26,2010-09-25,2011-09-24
AccountsPayableCurrent,NaN,NaN,12015000000,14632000000
AccountsReceivableNetCurrent,NaN,NaN,5510000000,5369000000
AccruedLiabilitiesCurrent,NaN,NaN,5723000000,9247000000
AccumulatedOtherComprehensiveIncomeLossNetOfTax,NaN,77000000,-46000000,443000000
Assets,NaN,47501000000,75183000000,116371000000
AssetsCurrent,NaN,NaN,41678000000,44988000000
AvailableForSaleSecuritiesCurrent,NaN,NaN,14359000000,16137000000
AvailableForSaleSecuritiesNoncurrent,NaN,NaN,25391000000,55618000000
CashAndCashEquivalentsAtCarryingValue,11875000000,5263000000,11261000000,9815000000
CommitmentsAndContingencies,NaN,NaN,0.0,0.0


In [54]:
example_income_statement = get_income_statement(get_xbrl_json(accession_numbers[10]))

In [58]:
example_income_one = example_income_statement.iloc[:, -4]

In [59]:
pd.DataFrame.to_csv(example_income_one, './data/apple_2011_income.csv')

In [17]:
loaded_income = pd.read_csv('./data/apple_2011_income.csv')

In [23]:
loaded_income.T


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Unnamed: 0,CostOfGoodsAndServicesSold,EarningsPerShareBasic,EarningsPerShareDiluted,GrossProfit,IncomeLossFromContinuingOperationsBeforeIncome...,IncomeTaxExpenseBenefit,NetIncomeLoss,NonoperatingIncomeExpense,OperatingExpenses,OperatingIncomeLoss,ResearchAndDevelopmentExpense,SalesRevenueNet,SellingGeneralAndAdministrativeExpense,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic
2010-09-26-2011-09-24,64431000000.0,28.05,27.68,43818000000.0,34205000000.0,8283000000.0,25922000000.0,415000000.0,10028000000.0,33790000000.0,2429000000.0,108249000000.0,7599000000.0,936645000.0,924258000.0


In [42]:
def get_cash_flow_statement(xbrl_json):
    cash_flows_store = {}

    for usGaapItem in xbrl_json['StatementsOfCashFlows']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfCashFlows'][usGaapItem]:        
            # only consider items without segment.
            if 'segment' not in fact:
                # check if date instant or date range is present
                if "instant" in fact['period']:
                    index = fact['period']['instant']
                else:
                    index = fact['period']['startDate'] + '-' + fact['period']['endDate']

                # avoid duplicate indicies with same values
                if index in indicies:
                    continue

                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    

        cash_flows_store[usGaapItem] = pd.Series(values, index=indicies) 


    cash_flows = pd.DataFrame(cash_flows_store)
    return cash_flows.T

In [6]:
example_cash_flow = get_cash_flow_statement(get_xbrl_json(accession_numbers[10]))

NameError: name 'accession_numbers' is not defined

In [ ]:
example_cash_flow_one = example_cash_flow.iloc[:, -5]

In [52]:
pd.DataFrame.to_csv(example_cash_flow_one, './data/apple_2011_cashflow.csv')

In [19]:
loaded_cash_flow = pd.read_csv('./data/apple_2011_cashflow.csv')

In [20]:
loaded_cash_flow

,Unnamed: 0,2010-09-26-2011-09-24
0,CashAndCashEquivalentsAtCarryingValue,NaN
1,CashAndCashEquivalentsPeriodIncreaseDecrease,-1.446000e+09
2,DeferredIncomeTaxExpenseBenefit,2.868000e+09
3,DepreciationAmortizationAndAccretionNet,1.814000e+09
4,EmployeeServiceShareBasedCompensationCashUsedT...,5.200000e+08
5,ExcessTaxBenefitFromShareBasedCompensationFina...,1.133000e+09
6,IncomeTaxesPaidNet,3.338000e+09
7,IncreaseDecreaseInAccountsPayable,2.515000e+09
8,IncreaseDecreaseInAccountsReceivable,-1.430000e+08
9,IncreaseDecreaseInDeferredRevenue,1.654000e+09


In [43]:
# convert XBRL-JSON of balance sheet to pandas dataframe
def get_balance_sheet(xbrl_json):
    balance_sheet_store = {}

    for usGaapItem in xbrl_json['BalanceSheets']:
        values = []
        indicies = []

        for fact in xbrl_json['BalanceSheets'][usGaapItem]:
            # only consider items without segment.
            if 'segment' not in fact:
                index = fact['period']['instant']

                # avoid duplicate indicies with same values
                if index in indicies:
                    continue
                    
                # add 0 if value is nil
                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    

            balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies) 

    balance_sheet = pd.DataFrame(balance_sheet_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date instant
    return balance_sheet

In [26]:
example_balance = get_balance_sheet(get_xbrl_json(accession_numbers[10]))

In [45]:
example_balance_T = pd.DataFrame(example_balance.iloc[:, -1]).T

In [53]:
pd.DataFrame.to_csv(example_balance_T, './data/apple_2011_balance.csv')

In [21]:
loaded_balance = pd.read_csv('./data/apple_2011_balance.csv')

In [22]:
loaded_balance.head()

,Unnamed: 0,AccountsPayableCurrent,AccountsReceivableNetCurrent,AccruedLiabilitiesCurrent,AccumulatedOtherComprehensiveIncomeLossNetOfTax,Assets,AssetsCurrent,AvailableForSaleSecuritiesCurrent,AvailableForSaleSecuritiesNoncurrent,CashAndCashEquivalentsAtCarryingValue,...,Liabilities,LiabilitiesAndStockholdersEquity,LiabilitiesCurrent,NontradeReceivablesCurrent,OtherAssetsCurrent,OtherAssetsNoncurrent,OtherLiabilitiesNoncurrent,PropertyPlantAndEquipmentAndCapitalizedSoftwareNet,RetainedEarningsAccumulatedDeficit,StockholdersEquity
0,2011-09-24,14632000000,5369000000,9247000000,443000000,116371000000,44988000000,16137000000,55618000000,9815000000,...,39756000000,116371000000,27970000000,6348000000,4529000000,3556000000,10100000000,7777000000,62841000000,76615000000


In [11]:
# clean income statement.
# drop duplicate columns (= column name ends with "_left"), drop key_0 column, drop columns with +5 NaNs
def clean_income_statement(statement):
    for column in statement:

        # column has more than 5 NaN values
        is_nan_column = statement[column].isna().sum() > 5

        if column.endswith('_left') or column == 'key_0' or is_nan_column:
            statement = statement.drop(column, axis=1)
    
    # rearrange columns so that first column represents first quarter
    # e.g. 2018, 2019, 2020 - and not 2020, 2019, 2018
    sorted_columns = sorted(statement.columns.values)
    
    return statement[sorted_columns]

In [12]:
# merge two income statements into one statement.
# row indicies of both statements have to be the same
# statement_b represents the most recent statement.
def merge_income_statements(statement_a, statement_b):
    return statement_a.merge(statement_b,
                     how="outer", 
                    #  on=statement_b.index, 
                    right_on=statement_b.index, 
                     left_index=True,
                    #  right_index=True,
                     suffixes=('_left', ''))

In [15]:
# helps printing dataframes while we generate new income statement
from IPython.display import display, HTML

previous_income_statement_set = False
income_statement_final = None

# XBRL-to-JSON converter API endpoint
xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"

for accession_no in accession_numbers[0:5]:
# for accession_no in accession_numbers: # doesn't work with filings filed before 2017 - indicies not equal
    print('Processing: ' + accession_no)
    
    # get XBRL-JSON of 10-Q or 10-K filing by accession number
    xbrl_json_data = get_xbrl_json(accession_no)
    
    # convert XBRL-JSON to a pandas dataframe
    income_statement_uncleaned = get_income_statement(xbrl_json_data)

    # clean the income statement
    income_statement_cleaned = clean_income_statement(income_statement_uncleaned)
    
    # print income statement on each iteration to monitor progress
    # display(HTML(income_statement_cleaned.to_html()))
    
    # merge new income statement with previously generated income statement
    if previous_income_statement_set:
        income_statement_final = clean_income_statement(merge_income_statements(income_statement_final, income_statement_cleaned))
    else:
        income_statement_final = income_statement_cleaned
        previous_income_statement_set = True

Processing: 0001144204-11-072464


KeyError: 'StatementsOfIncome'

In [21]:
income_statement_final.T

,NaN,CostOfGoodsAndServicesSold,GrossProfit,ResearchAndDevelopmentExpense,SellingGeneralAndAdministrativeExpense,OperatingExpenses,OperatingIncomeLoss,NonoperatingIncomeExpense,IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,IncomeTaxExpenseBenefit,NetIncomeLoss,EarningsPerShareBasic,EarningsPerShareDiluted,WeightedAverageNumberOfSharesOutstandingBasic,WeightedAverageNumberOfDilutedSharesOutstanding,NaN,SalesRevenueNet,CommonStockDividendsPerShareDeclared
2014-09-28-2015-09-26,NaN,140089000000,93626000000,8067000000,14329000000,22396000000,71230000000,1285000000,72515000000,19121000000,53394000000,9.28,9.22,5753421000,5793069000,NaN,233715000000,1.98
2015-09-27-2016-09-24,NaN,131376000000,84263000000,10045000000,14194000000,24239000000,60024000000,1348000000,61372000000,15685000000,45687000000,8.35,8.31,5470820000,5500281000,NaN,215639000000,2.18
2016-09-25-2017-09-30,NaN,141048000000,88186000000,11581000000,15261000000,26842000000,61344000000,2745000000,64089000000,15738000000,48351000000,9.27,9.21,5217242000,5251692000,NaN,229234000000,2.4
2017-10-01-2018-09-29,NaN,163756000000,101839000000,14236000000,16705000000,30941000000,70898000000,2005000000,72903000000,13372000000,59531000000,12.01,11.91,4955377000,5000109000,265595000000,NaN,NaN
2018-09-30-2019-09-28,260174000000,161782000000,98392000000,16217000000,18245000000,34462000000,63930000000,1807000000,65737000000,10481000000,55256000000,11.97,11.89,4617834000,4648913000,NaN,NaN,NaN
2019-09-29-2020-09-26,274515000000,169559000000,104956000000,18752000000,19916000000,38668000000,66288000000,803000000,67091000000,9680000000,57411000000,3.31,3.28,17352119000,17528214000,NaN,NaN,NaN
2020-09-27-2021-09-25,365817000000,212981000000,152836000000,21914000000,21973000000,43887000000,108949000000,258000000,109207000000,14527000000,94680000000,5.67,5.61,16701272000,16864919000,NaN,NaN,NaN


In [49]:
cleaned_list_xbrl[6]

{'CoverPage': {'EntityRegistrantName': 'SINGLE TOUCH SYSTEMS INC',
  'DocumentType': '10-K',
  'DocumentPeriodEndDate': '2011-09-30',
  'AmendmentFlag': 'false',
  'EntityCentralIndexKey': '0001157817',
  'CurrentFiscalYearEndDate': '--09-30',
  'EntityCommonStockSharesOutstanding': {'unitRef': 'Shares',
   'decimals': 'INF',
   'period': {'instant': '2011-09-30'},
   'value': '130182392'},
  'EntityPublicFloat': {'unitRef': 'USD',
   'decimals': '0',
   'period': {'instant': '2011-09-30'},
   'value': '67573482'},
  'EntityFilerCategory': 'Smaller Reporting Company',
  'EntityCurrentReportingStatus': 'No',
  'EntityVoluntaryFilers': 'No',
  'EntityWellKnownSeasonedIssuer': 'No',
  'DocumentFiscalYearFocus': '2011',
  'DocumentFiscalPeriodFocus': 'FY'},
 'BalanceSheets': {'CashAndCashEquivalentsAtCarryingValue': [{'unitRef': 'USD',
    'decimals': '0',
    'period': {'instant': '2011-09-30'},
    'value': '523801'},
   {'unitRef': 'USD',
    'decimals': '0',
    'period': {'instant': '

In [109]:
# convert XBRL-JSON of balance sheet to pandas dataframe
def get_balance_sheet(xbrl_json):
    balance_sheet_store = {}

    for usGaapItem in xbrl_json['BalanceSheets']:
        values = []
        indicies = []

        for fact in xbrl_json['BalanceSheets'][usGaapItem]:
            # only consider items without segment.
            if 'segment' not in fact:
                index = fact['period']['instant']

                # avoid duplicate indicies with same values
                if index in indicies:
                    continue
                    
                # add 0 if value is nil
                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    

            balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies) 

    balance_sheet = pd.DataFrame(balance_sheet_store).reset_index()
    balance_sheet['index'] = pd.to_datetime(balance_sheet['index'])
    balance_sheet['year'] = pd.DatetimeIndex(balance_sheet['index']).year
    balance_sheet['CIK'] = xbrl_json['CoverPage']['EntityCentralIndexKey']
    balance_sheet = pd.DataFrame(balance_sheet.iloc[-1, :]).T.drop('index', axis=1)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date instant
    return balance_sheet

In [110]:
test = get_balance_sheet(cleaned_list_xbrl[35])
test

,CashAndCashEquivalentsAtCarryingValue,AccountsReceivableNetCurrent,InventoryNet,PrepaidExpenseAndOtherAssetsCurrent,AssetsCurrent,PropertyPlantAndEquipmentNet,IntangibleAssetsNetExcludingGoodwill,Assets,AccountsPayableCurrent,AccruedLiabilitiesCurrent,...,LongTermNotesPayable,Liabilities,PreferredStockValue,CommonStockValue,AdditionalPaidInCapital,RetainedEarningsAccumulatedDeficit,StockholdersEquity,LiabilitiesAndStockholdersEquity,year,CIK
2,4356,25922,6286,1020,37584,760,93256,131600,1697987,2158796,...,0,6881234,1381849,68091,19831800,-28031374,-6749634,131600,2011,0001031896


In [111]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 2 to 2
Data columns (total 25 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   CashAndCashEquivalentsAtCarryingValue        1 non-null      object
 1   AccountsReceivableNetCurrent                 1 non-null      object
 2   InventoryNet                                 1 non-null      object
 3   PrepaidExpenseAndOtherAssetsCurrent          1 non-null      object
 4   AssetsCurrent                                1 non-null      object
 5   PropertyPlantAndEquipmentNet                 1 non-null      object
 6   IntangibleAssetsNetExcludingGoodwill         1 non-null      object
 7   Assets                                       1 non-null      object
 8   AccountsPayableCurrent                       1 non-null      object
 9   AccruedLiabilitiesCurrent                    1 non-null      object
 10  NotesPayableCurren

In [112]:
test2 = get_balance_sheet(cleaned_list_xbrl[10])
test2

,Cash,AccountsReceivableNetCurrent,AssetsCurrent,PropertyPlantAndEquipmentNet,Assets,AccountsPayableAndAccruedLiabilitiesCurrent,NotesPayableCurrent,Liabilities,CommonStockValue,AdditionalPaidInCapitalCommonStock,DevelopmentStageEnterpriseDeficitAccumulatedDuringDevelopmentStage,StockholdersEquity,LiabilitiesAndStockholdersEquity,year,CIK
4,17998,6328,24326,1970,26296,132946,27780,160726,3895,65080,203405,-134430,26296,2011,0001482179


In [113]:
pd.concat([test, test2])

,CashAndCashEquivalentsAtCarryingValue,AccountsReceivableNetCurrent,InventoryNet,PrepaidExpenseAndOtherAssetsCurrent,AssetsCurrent,PropertyPlantAndEquipmentNet,IntangibleAssetsNetExcludingGoodwill,Assets,AccountsPayableCurrent,AccruedLiabilitiesCurrent,...,AdditionalPaidInCapital,RetainedEarningsAccumulatedDeficit,StockholdersEquity,LiabilitiesAndStockholdersEquity,year,CIK,Cash,AccountsPayableAndAccruedLiabilitiesCurrent,AdditionalPaidInCapitalCommonStock,DevelopmentStageEnterpriseDeficitAccumulatedDuringDevelopmentStage
2,4356,25922,6286,1020,37584,760,93256,131600,1697987,2158796,...,19831800,-28031374,-6749634,131600,2011,0001031896,NaN,NaN,NaN,NaN
4,NaN,6328,NaN,NaN,24326,1970,NaN,26296,NaN,NaN,...,NaN,NaN,-134430,26296,2011,0001482179,17998,132946,65080,203405


In [116]:
full_list = []

for xbrl in cleaned_list_xbrl:
    try:
        full_list.append(get_balance_sheet(xbrl))
    except:
        continue
len(full_list)    
balance_full_df = pd.concat(full_list)
balance_full_df.head()

/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel_10886/1287799126.py:26: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies)
/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel_10886/1287799126.py:26: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies)
/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel_10886/1287799126.py:26: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies)
/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel

,CashAndCashEquivalentsAtCarryingValue,NotesAndLoansReceivableNetCurrent,PrepaidExpenseAndOtherAssetsCurrent,AssetsCurrent,PropertyPlantAndEquipmentNet,Assets,AccountsPayableCurrent,EmployeeRelatedLiabilitiesCurrent,OtherLiabilitiesCurrent,AdvancesOnStockSubscriptions,...,UnrecoveredCostsForPurchasedGasAmount,DeferredCosts,UnamortizedDebtIssuanceExpense,OtherShortTermBorrowings,DepositLiabilitiesAccruedInterest,DividendsPayableCurrentAndNoncurrent,DeferredCompensationLiabilityClassifiedNoncurrent,PensionAndOtherPostretirementDefinedBenefitPlansLiabilitiesNoncurrent,DueToRelatedParties,OtherNotesPayable
2,5097,47500,29299,81896,13273,95169,1794935,771473,573667,142200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,453243,NaN,NaN,714818,3964003,5545052,180031,NaN,75056,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2124406,NaN,NaN,20916648,5155528,30596000,3233597,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39188,NaN,8656,93842,12201,106043,333735,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,75780,43068,204023,167420,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
balance_full_df.columns

Index(['CashAndCashEquivalentsAtCarryingValue',
       'NotesAndLoansReceivableNetCurrent',
       'PrepaidExpenseAndOtherAssetsCurrent', 'AssetsCurrent',
       'PropertyPlantAndEquipmentNet', 'Assets', 'AccountsPayableCurrent',
       'EmployeeRelatedLiabilitiesCurrent', 'OtherLiabilitiesCurrent',
       'AdvancesOnStockSubscriptions',
       ...
       'UnrecoveredCostsForPurchasedGasAmount', 'DeferredCosts',
       'UnamortizedDebtIssuanceExpense', 'OtherShortTermBorrowings',
       'DepositLiabilitiesAccruedInterest',
       'DividendsPayableCurrentAndNoncurrent',
       'DeferredCompensationLiabilityClassifiedNoncurrent',
       'PensionAndOtherPostretirementDefinedBenefitPlansLiabilitiesNoncurrent',
       'DueToRelatedParties', 'OtherNotesPayable'],
      dtype='object', length=164)

In [120]:
balance_full_df[['year', 'CIK', 'AssetsCurrent', 'LiabilitiesCurrent']]

,year,CIK,AssetsCurrent,LiabilitiesCurrent
2,2011,0001162816,81896,3638136
2,2011,0001497649,714818,255087
3,2011,0000718924,20916648,5604885
2,2011,0001362516,93842,670494
15,2011,0001039466,75780,729821
3,2011,0000068709,324056000,195614000
5,2011,0001112372,400964,1476517
4,2011,0001482179,24326,NaN
3,2011,0000808326,91351000,67058000
5,2011,0001383534,NaN,NaN


In [118]:
column_names = [column for column in balance_full_df.columns]

In [119]:
sorted(column_names)

['AccountsAndOtherReceivablesNetCurrent',
 'AccountsPayableAndAccruedLiabilitiesCurrent',
 'AccountsPayableCurrent',
 'AccountsPayableTradeCurrent',
 'AccountsReceivableGrossCurrent',
 'AccountsReceivableNetCurrent',
 'AccruedExpensesAndOtherCurrentLiabilities',
 'AccruedIncomeTaxesCurrent',
 'AccruedIncomeTaxesNoncurrent',
 'AccruedLiabilitiesCurrent',
 'Accruedexpensesandotherpayable',
 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment',
 'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax',
 'AccumulatedOtherComprehensiveIncomeLossNetOfTax',
 'AdditionalPaidInCapital',
 'AdditionalPaidInCapitalCommonStock',
 'AdvancesOnStockSubscriptions',
 'AssetRetirementObligation',
 'AssetRetirementObligationsNoncurrent',
 'Assets',
 'AssetsCurrent',
 'AssetsNoncurrent',
 'AssetsNoncurrentExcludingPropertyPlantAndEquipment',
 'AvailableForSaleSecuritiesCurrent',
 'AvailableForSaleSecuritiesNoncurrent',
 'BillingsInExcessOfCostCurrent',
 'B

In [100]:
balance_test_drop = balance_full_df.dropna(axis=1)

In [103]:
pd.set_option('display.max_rows', None)

In [106]:
balance_full_df.isna().sum() 

index                                                                                         0
CashAndCashEquivalentsAtCarryingValue                                                         5
NotesAndLoansReceivableNetCurrent                                                            24
PrepaidExpenseAndOtherAssetsCurrent                                                          15
AssetsCurrent                                                                                 2
PropertyPlantAndEquipmentNet                                                                  4
Assets                                                                                        1
AccountsPayableCurrent                                                                        6
EmployeeRelatedLiabilitiesCurrent                                                            23
OtherLiabilitiesCurrent                                                                      19
AdvancesOnStockSubscriptions            

In [84]:
balance_test_drop

,index,LiabilitiesAndStockholdersEquity,CIK
2,2011-09-30,95169,0001162816
2,2011-09-30,5545052,0001497649
3,2011-10-01,30596000,0000718924
2,2011-09-30,106043,0001362516
15,2011-09-30,204023,0001039466
3,2011-10-01,427859000,0000068709
5,2011-09-30,1021325,0001112372
4,2011-09-30,26296,0001482179
3,2011-09-30,170298000,0000808326
5,2011-09-30,5718,0001383534


In [2]:
tsla = yf.Ticker('TSLA')

In [7]:
tsla.financials

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Research Development,2593000000.0,1491000000.0,1343000000.0,1460000000.0
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,6343000000.0,1154000000.0,-665000000.0,-1005000000.0
Minority Interest,1394000000.0,1454000000.0,1492000000.0,1390000000.0
Net Income,5519000000.0,721000000.0,-862000000.0,-976000000.0
Selling General Administrative,4517000000.0,3188000000.0,2646000000.0,2835000000.0
Gross Profit,13606000000.0,6630000000.0,4069000000.0,4042000000.0
Ebit,6523000000.0,1951000000.0,80000000.0,-253000000.0
Operating Income,6523000000.0,1951000000.0,80000000.0,-253000000.0
Other Operating Expenses,-27000000.0,-27000000.0,-27000000.0,-27000000.0


In [126]:
#### convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    

        income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 

    income_statement = pd.DataFrame(income_statement_store).reset_index()
    income_statement['index'] = pd.to_datetime(income_statement['index'])
    income_statement['year'] = pd.DatetimeIndex(income_statement['index']).year
    income_statement['CIK'] = xbrl_json['CoverPage']['EntityCentralIndexKey']
    income_statement = pd.DataFrame(income_statement.iloc[-1, :]).T.drop('index', axis=1)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return income_statement



In [127]:
get_income_statement(cleaned_list_xbrl[9])

,SalesRevenueGoodsGross,CostOfGoodsSold,GrossProfit,SellingGeneralAndAdministrativeExpense,OperatingExpenses,OperatingIncomeLoss,MinorityInterestInNetIncomeLossJointVenturePartners,AmortizationOfIntangibleAssets,NonoperatingGainsLosses,InterestExpenseDebt,NonoperatingIncomeExpense,IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,IncomeTaxExpenseBenefit,ProfitLoss,EarningsPerShareBasic,WeightedAverageCommonSharesOutstandingBasicAndDiluted,year,CIK
3,437378,200845,236533,702169,702169,-465636,130762,0,-130762,47041,-47041,-643439,0,-643439,-0.01,69736503,2010,0001112372


In [60]:
cleaned_list_xbrl[0]

{'CoverPage': {'EntityRegistrantName': 'NEAH POWER SYSTEMS, INC.',
  'DocumentType': '10-K',
  'DocumentPeriodEndDate': '2011-09-30',
  'AmendmentFlag': 'false',
  'EntityCentralIndexKey': '0001162816',
  'CurrentFiscalYearEndDate': '--09-30',
  'EntityCommonStockSharesOutstanding': {'unitRef': 'Shares',
   'decimals': 'INF',
   'period': {'instant': '2011-12-23'},
   'value': '182793529'},
  'EntityPublicFloat': {'unitRef': 'USD',
   'decimals': 'INF',
   'period': {'instant': '2011-03-31'},
   'value': '800000'},
  'EntityFilerCategory': 'Smaller Reporting Company',
  'EntityCurrentReportingStatus': 'Yes',
  'EntityVoluntaryFilers': 'No',
  'EntityWellKnownSeasonedIssuer': 'No',
  'DocumentFiscalYearFocus': '2011',
  'DocumentFiscalPeriodFocus': 'FY'},
 'StatementsOfIncome': {'Revenues': [{'xsi:nil': 'true',
    'unitRef': 'USD',
    'period': {'startDate': '2010-10-01', 'endDate': '2011-09-30'}},
   {'xsi:nil': 'true',
    'unitRef': 'USD',
    'period': {'startDate': '2009-10-01', 

In [128]:
full_list2 = []

for xbrl in cleaned_list_xbrl:
    try:
        full_list2.append(get_income_statement(xbrl))
    except:
        continue
len(full_list2)    


/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel_10886/1910821255.py:19: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  income_statement_store[usGaapItem] = pd.Series(values, index=indicies)
/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel_10886/1910821255.py:19: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  income_statement_store[usGaapItem] = pd.Series(values, index=indicies)
/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel_10886/1910821255.py:19: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  income_statement_store[usGaapItem] = pd.Series(values, index=indicies)
/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/

16

In [129]:
income_full_df = pd.concat(full_list2)

In [130]:
income_full_df

,OilAndGasSalesRevenue,LeaseOperatingExpense,ProductionTaxExpense,DepreciationDepletionAndAmortization,AssetRetirementObligationAccretionExpense,GeneralAndAdministrativeExpense,OtherExpenses,CostsAndExpenses,OperatingIncomeLoss,InvestmentIncomeInterest,...,OtherNonoperatingExpense,InvestmentIncomeNonoperating,OtherIncome,RentalIncomeNonoperating,DeferredIncomeTaxExpenseBenefit,OtherComprehensiveIncomeLossNetOfTaxPeriodIncreaseDecrease,AssetImpairmentCharges,ImpairmentOfIntangibleAssetsFinitelived,LitigationLossProvision,GainLossOnSaleOfPropertyPlantEquipment
1,0,0,0,0,0,107033,11111,118144,-118144,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2632453000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-465636,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,70243,NaN,NaN,-44225,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-32527000,2000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2243835,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-178949160,2283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61203000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2597000,5000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,54105,NaN,NaN,-54105,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
income_full_df[['OperatingIncomeLoss', 'NetIncomeLoss', 'CIK', 'year']]

,OperatingIncomeLoss,NetIncomeLoss,CIK,year
1,-118144,-118144,0001497649,2010
2,NaN,-716712000,0000357294,2008
3,-465636,NaN,0001112372,2010
4,-44225,-107829,0001482179,2010
2,-32527000,-34219000,0000808326,2010
1,2243835,1348734,0000030831,2010
10,-178949160,-178921042,0001135202,2010
2,61203000,16491000,0001512574,2010
1,2597000,259000,0000017485,2010
2,-54105,-54105,0001502152,2010


In [166]:
def get_cash_flow_statement(xbrl_json):
    cash_flows_store = {}

    for usGaapItem in xbrl_json['StatementsOfCashFlows']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfCashFlows'][usGaapItem]:        
            # only consider items without segment.
            if 'segment' not in fact:
                # check if date instant or date range is present
                if "instant" in fact['period']:
                    index = fact['period']['instant']
                else:
                    index = fact['period']['startDate'] + '-' + fact['period']['endDate']

                # avoid duplicate indicies with same values
                if index in indicies:
                    continue

                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    

        cash_flows_store[usGaapItem] = pd.Series(values, index=indicies) 


    cash_flows = pd.DataFrame(cash_flows_store).reset_index()
    cash_flows['index'] = cash_flows['index'].astype('str').replace('-', '')
    #cash_flows['year'] = pd.DatetimeIndex(cash_flows['index']).year
    #cash_flows['CIK'] = xbrl_json['CoverPage']['EntityCentralIndexKey']
    #cash_flows = pd.DataFrame(cash_flows.iloc[-2, :]).T.drop('index', axis=1)
    return cash_flows

In [167]:
get_cash_flow_statement(cleaned_list_xbrl[20])

,index,NetIncomeLoss,DepreciationDepletionAndAmortization,ShareBasedCompensation,ProvisionForLoanLeaseAndOtherLosses,AmortizationOfDebtDiscountPremium,EmbeddedDerivativeGainLossOnEmbeddedDerivativeNet,DeferredIncomeTaxExpenseBenefit,AmortizationOfFinancingCosts,InventoryWriteDown,...,ProceedsFromStockPlans,PaymentsForRepurchaseOfCommonStock,RepaymentsOfDebtAndCapitalLeaseObligations,NetCashProvidedByUsedInFinancingActivities,CashPeriodIncreaseDecrease,Cash,InterestPaid,IncomeTaxesPaid,PaidInKindInterest,DebtConversionConvertedInstrumentAmount1
0,2009-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3798000,NaN,NaN,NaN,NaN
1,2009-10-01-2010-09-30,2600000,5150000,388000,3096000,1003000,-691000,130000,26000,0,...,128000,9000,1031000,7273000,3630000,NaN,337000,13000,45000,8913000
2,2010-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7428000,NaN,NaN,NaN,NaN
3,2010-10-01-2011-09-30,259000,8790000,385000,3950000,21000,-902000,1340000,0,35000,...,86000,0,621000,914000,-4412000,NaN,56000,-8000,0,5100000
4,2011-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3016000,NaN,NaN,NaN,NaN
